In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly 
import plotly.graph_objects as go
import plotly.express as px
%matplotlib qt
pd.set_option('mode.chained_assignment', None)

# Load raw and save ready data

In [36]:
# path = f'Experiments\MultiplyTemperature\Exp5(12.5)\Exp5_down.csv'

path=''
while (path == '') or (not os.path.isfile(path)):
    path = input(f"Input data path: ")
_path_list = (path).split('\\')
folder_path = '\\'.join(_path_list[:-1])
data_name = _path_list[-1]
print(f'    Folder:\n{folder_path}\n    Data name:\n {data_name}')

    Folder:
Experiments\MultiplyTemperature\Exp6(15)
    Data name:
 Exp6_down.csv


In [37]:
df1 = pd.read_csv(path,index_col=0)
# df2 = pd.read_csv(f'{path}/Exp0_up2.csv',index_col=0)
# df3 = pd.read_csv(f'{path}/Exp0_up3.csv',index_col=0)
# df4 = pd.read_csv(f'{path}/Exp0_up4.csv',index_col=0)

In [38]:
df1['Viscosity_verbose'].unique()

array(['OK', 'error', 'image_sweep_check', 'OK_inner'], dtype=object)

In [39]:
rules = {
    'processor_sweep':'image_sweep_check',
    'value_combine': 'combine_check',
    'inner_processor_check':'OK_inner'
}
col_rules = {
    'Viscosity_mark': 'Viscosity_verbose',
    'Temperature_mark': 'Temperature_verbose',
}

df1 = df1.rename(columns=col_rules)
df1[['Viscosity_verbose','Temperature_verbose']] =\
    df1[['Viscosity_verbose','Temperature_verbose']].replace(rules)

# df2 = df2.rename(columns=col_rules)
# df2[['Viscosity_verbose','Temperature_verbose']] = df2[['Viscosity_verbose','Temperature_verbose']].replace(rules)
# df2['time'] =df2['time']+ df1['time'].max()+1

# df3 = df3.rename(columns=col_rules)
# df3[['Viscosity_verbose','Temperature_verbose']] = df3[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df3['time'] =df3['time']+ df2['time'].max()+1

# df4 = df4.rename(columns=col_rules)
# df4[['Viscosity_verbose','Temperature_verbose']] = df4[['Viscosity_verbose','Temperature_verbose']].replace({np.nan:'OK'})
# df4['time'] =df4['time']+ df3['time'].max()+1
# df = pd.concat([df1,df2,df3,df4])

df = df1.dropna(subset=['Viscosity', 'Temperature'])
df

,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
0,0.0,16.0,OK,42.1,OK
1,2.0,15.0,OK,42.1,OK
2,4.0,16.0,OK,42.1,OK
3,6.0,16.0,OK,42.1,OK
4,8.0,16.0,OK,42.1,OK
...,...,...,...,...,...
2480,4960.0,60.1,image_sweep_check,14.8,OK
2481,4962.0,60.0,OK,14.8,OK
2482,4964.0,60.0,OK,14.8,OK
2483,4966.0,60.0,OK,14.8,OK


In [40]:
np.any(df['time'].duplicated())

False

In [41]:
ready_data_name = f'AData_{data_name}'


assert not os.path.isfile(f'{folder_path}\{ready_data_name}'), 'Data already exists'
df.to_csv(f'{folder_path}\{ready_data_name}',index=0)

# Load

In [63]:
data_path=''
while (data_path == '') or (not os.path.isfile(data_path)):
    data_path = input(f"Input data path: ")
df = pd.read_csv(data_path)
df

,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
0,0.0,38.2,OK,38.1,image_sweep_check
1,1.0,38.2,OK,38.1,OK
2,2.0,38.2,OK,38.1,OK
3,3.0,38.1,image_sweep_check,38.2,OK
4,4.0,38.0,OK_inner,38.2,OK
...,...,...,...,...,...
7530,7674.0,155.4,image_sweep_check,14.9,OK
7531,7675.0,159.3,OK,14.9,OK
7532,7676.0,159.3,image_sweep_check,14.9,OK
7533,7677.0,157.8,OK_inner,14.9,OK


# Plots

In [42]:
temperature_cond = ((12 < df['Temperature']) & (df['Temperature'] < 45))
viscosity_cond = ((0 < df['Viscosity']) & (df['Viscosity'] < 500))

dfs = df[temperature_cond & viscosity_cond]
# dfs =df

In [43]:
# plot_rules = {'processor_sweep':2, 'OK':0, 'error':4, 'value_combine':3,
#        'inner_processor_check':1}
# df[['Viscosity_verbose','Temperature_verbose']] = df[['Viscosity_verbose','Temperature_verbose']].replace(plot_rules)
# dfs['time'] = dfs['time'] / 60


In [44]:
# T_goupT= dfs.groupby(by='Temperature')['Temperature']
# T_groupV=dfs.groupby(by='Temperature')['Viscosity']
# def foo(g):
#     whisker_width=0
#     q1 = g.quantile(0.45)                
#     q3 = g.quantile(0.55)
#     iqr = q3 - q1
#     return (g >= q1 - whisker_width*iqr) & (g <= q3 + whisker_width*iqr)
#     # return np.abs((g - g.median()) / iqr) < 2.22
#     # return np.abs((g - g.mean()) / g.std(ddof=0)) < 1

# error_mask= T_groupV.apply(foo ).droplevel([0]).reset_index(drop=True).to_numpy()

# dfs = dfs[error_mask]


# # T_goupT2= dfs2.groupby(by='Temperature')['Temperature']
# # T_groupV2=dfs2.groupby(by='Temperature')['Viscosity']
# # dfs2
# dfs

In [45]:
dfs_median = dfs[['Temperature','Viscosity']].groupby(by='Temperature').median()

## Time plots

In [46]:
fig,ax_v =plt.subplots()
ax_T = ax_v.twinx()

ax_v.scatter(dfs['time'],dfs['Viscosity'],color='red',marker='.')
ax_v.set_ylabel('Viscosity',color='red')

ax_T.scatter(dfs['time'],dfs['Temperature'],color='blue',marker='.')
ax_T.set_ylabel('Temperature',color='blue')


# df_masked = dfs[(dfs['Temperature_verbose']>=3) | (dfs ['Viscosity_verbose']>=3)]
# ax_v.scatter(df_masked['time'],df_masked['Viscosity'],color='black',marker='s')
# ax_T.scatter(df_masked['time'],df_masked['Temperature'],color='gray',marker='o')

Text(0, 0.5, 'Temperature')

## Temterature plots

In [23]:
fig = go.Figure()
fig = px.scatter(
    dfs,
    x="Temperature",
    y="Viscosity",
    color='Viscosity_verbose',
)

fig.add_traces([
    go.Scatter(name='Median',
               x=dfs_median.index,
               y=dfs_median['Viscosity'],
               mode='lines'),
    # go.Scatter(name='Mean', x=T_goupT.max(), y=T_groupV.mean(), mode='lines'),
    # go.Scatter(
    #     name='Errors',
    #     x=T_goupT2.max(),
    #     y=T_groupV2.mean(),
    #     mode='markers',
    # ),
])
fig.update_layout(xaxis=dict(range=(13.5, 40.5)),
                  yaxis=dict(range=(0, 150)),
                  margin=dict(l=10, r=10, b=10, t=10, pad=0))
fig.show()